# IV - Parameter Sweep
In this notebook we will be running a simple parameter sweep on the model we have. We will then pull the results of our sweep and based on the results of our sweep pull the best performing model from blob.

* [Setup](#section1)
* [Configure job](#section2)
* [Submit job](#section3)
* [Check results](#section4)
* [Download best model](#section5)
* [Delete job](#section6)

<a id='section1'></a>

## Setup

Create a simple alias for Batch Shipyard

In [1]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working

In [2]:
shipyard

Usage: shipyard.py [OPTIONS] COMMAND [ARGS]...

  Batch Shipyard: Provision and Execute Docker Workloads on Azure Batch

Options:
  --version   Show the version and exit.
  -h, --help  Show this message and exit.

Commands:
  cert      Certificate actions
  data      Data actions
  fs        Filesystem in Azure actions
  jobs      Jobs actions
  keyvault  KeyVault actions
  misc      Miscellaneous actions
  pool      Pool actions
  storage   Storage actions


Read in the account information we saved earlier

In [3]:
import json

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)
    
account_info = read_json('account_information.json')

storage_account_key = account_info['storage_account_key']
storage_account_name = account_info['storage_account_name']
IMAGE_NAME = account_info['IMAGE_NAME']
STORAGE_ALIAS = account_info['STORAGE_ALIAS']

<a id='section2'></a>

## Configure Job

As in the previous job we ran on a single node we will be running the job on GPU enabled nodes. The difference here is that depending on the number of combinations we will be creating the same number of tasks. Each task will have a different set of parmeters that we will be passing to our model training script. This parameters effect the training of the model and in the end the performance of the model. The model and results of its evaluation are recorded and stored on the node. At the end of the task the results are pulled into the specified storage container..

In [4]:
from copy import copy
import os
import random
from sklearn.grid_search import ParameterGrid
from toolz import curry, pipe

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

def compose_command(num_convolution_layers, minibatch_size, max_epochs=30):
    cmd_str = ' '.join(("source /cntk/activate-cntk;",
                        "python -u ConvNet_CIFAR10.py",
                        "--datadir $AZ_BATCH_NODE_SHARED_DIR/data",
                        "--num_convolution_layers {num_convolution_layers}",
                        "--minibatch_size {minibatch_size}",
                        "--max_epochs {max_epochs}")).format(num_convolution_layers=num_convolution_layers,
                                                             minibatch_size=minibatch_size,
                                                             max_epochs=max_epochs)
    return 'bash -c "{}"'.format(cmd_str)

@curry
def append_parameter(param_name, param_value, data_dict):
    data_dict[param_name]=param_value
    return data_dict

def task_generator(parameters):
    for params in ParameterGrid(parameters):
        yield pipe(copy(_TASK_TEMPLATE),
                   append_parameter('command', compose_command(**params)))

/home/nbuser/anaconda2_410/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/nbuser/anaconda2_410/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Generate the `jobs.json` configuration file

In [5]:
_TASK_TEMPLATE = {
    "image": IMAGE_NAME,
    "remove_container_after_exit": True,
    "gpu": True,
    "resource_files": [
            {
                "file_path": "ConvNet_CIFAR10.py",
                "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/ConvNet_CIFAR10.py",
                "file_mode":'0777'
            }
    ],
    "output_data": {
        "azure_storage": [
            {
                "storage_account_settings": STORAGE_ALIAS,
                "container": "output",
                "source": "$AZ_BATCH_TASK_DIR/wd/Models"
            },
        ]
    },
}

For the purposes of the workshop, we are constraining the parameter search space to just 3 final combinations (4 will be generated but we will remove the first one such that we only need to wait for "one round" of processing. In other words, since we have 3 total compute nodes in the pool and 3 tasks, we only have to wait for "one round."

In [6]:
parameters = {
    "num_convolution_layers": [2, 3],  # this could be expanded to [2, 3, 4] for example
    "minibatch_size": [32, 64]         # this could be expanded to [32, 64, 128] for example
}

In [7]:
JOB_ID = 'cntk-parametricsweep-job'

jobs = {
    "job_specifications": [
        {
            "id": JOB_ID,
            "tasks": list(task_generator(parameters))    
        }
    ]
}

# for purposes of expediency in the workshop, we'll remove one of the tasks to
# make 3 total to match the number of compute nodes in our pool
del jobs['job_specifications'][0]['tasks'][0]

print('number of tasks for parametric sweep {}: {}'.format(JOB_ID, len(jobs['job_specifications'][0]['tasks'])))

number of tasks for parametric sweep cntk-parametricsweep-job: 3


In [8]:
write_json_to_file(jobs, os.path.join('config', 'jobs.json'))
print(json.dumps(jobs, indent=4, sort_keys=True))

{
    "job_specifications": [
        {
            "id": "cntk-parametricsweep-job", 
            "tasks": [
                {
                    "command": "bash -c \"source /cntk/activate-cntk; python -u ConvNet_CIFAR10.py --datadir $AZ_BATCH_NODE_SHARED_DIR/data --num_convolution_layers 3 --minibatch_size 32 --max_epochs 30\"", 
                    "gpu": true, 
                    "image": "microsoft/cntk:2.0-gpu-python3.5-cuda8.0-cudnn5.1", 
                    "output_data": {
                        "azure_storage": [
                            {
                                "container": "output", 
                                "source": "$AZ_BATCH_TASK_DIR/wd/Models", 
                                "storage_account_settings": "mystorageaccount"
                            }
                        ]
                    }, 
                    "remove_container_after_exit": true, 
                    "resource_files": [
                        {
                       

<a id='section3'></a>

## Submit job
Check that everything is ok with our pool before we submit our jobs

In [9]:
shipyard pool listnodes

2017-07-12 14:04:36,121 DEBUG - listing nodes for pool gpupool
2017-07-12 14:04:36,424 INFO - node_id=tvm-1392786932_1-20170712t132611z [state=ComputeNodeState.idle start_task_exit_code=0 scheduling_state=SchedulingState.enabled ip_address=10.0.0.6 vm_size=standard_nc6 dedicated=True total_tasks_run=0 running_tasks_count=0 total_tasks_succeeded=0]
2017-07-12 14:04:36,424 INFO - node_id=tvm-1392786932_2-20170712t132611z [state=ComputeNodeState.idle start_task_exit_code=0 scheduling_state=SchedulingState.enabled ip_address=10.0.0.5 vm_size=standard_nc6 dedicated=True total_tasks_run=0 running_tasks_count=0 total_tasks_succeeded=0]
2017-07-12 14:04:36,424 INFO - node_id=tvm-1392786932_3-20170712t132611z [state=ComputeNodeState.idle start_task_exit_code=0 scheduling_state=SchedulingState.enabled ip_address=10.0.0.4 vm_size=standard_nc6 dedicated=True total_tasks_run=2 running_tasks_count=0 total_tasks_succeeded=2]


Now that we have confirmed everything is working we can execute our job using the command below. 

In [10]:
shipyard jobs add

2017-07-12 14:04:40,170 INFO - Adding job cntk-parametricsweep-job to pool gpupool
2017-07-12 14:04:40,975 INFO - uploading file /tmp/tmpdpLBXc as u'shipyardtaskrf-cntk-parametricsweep-job/dockertask-00000.shipyard.envlist'
2017-07-12 14:04:41,500 INFO - uploading file /tmp/tmp0e9zXa as u'shipyardtaskrf-cntk-parametricsweep-job/dockertask-00001.shipyard.envlist'
2017-07-12 14:04:42,225 INFO - uploading file /tmp/tmpVymWG5 as u'shipyardtaskrf-cntk-parametricsweep-job/dockertask-00002.shipyard.envlist'
2017-07-12 14:04:42,453 DEBUG - submitting 3 tasks (0 -> 2) to job cntk-parametricsweep-job
2017-07-12 14:04:42,750 INFO - submitted all 3 tasks to job cntk-parametricsweep-job


Using the command below we can check the status of our job. Only after all tasks have an exit code can we continue with the notebook. Please keep re-running the cell below periodically until you see that all tasks show completed state with an exit code. Continuing on with the notebook without all tasks in the job completing their training execution will result in failure in subsequent cells.

You can also view the **heatmap** of this pool on [Azure Portal](https://portal.azure.com) to monitor the progress of this job on the compute nodes under your Batch account.

In [17]:
shipyard jobs listtasks --jobid $JOB_ID

2017-07-12 14:17:58,284 INFO - job_id=cntk-parametricsweep-job task_id=dockertask-00000 [state=TaskState.completed max_retries=0 retention_time=10675199 days, 2:48:05.477581 pool_id=gpupool node_id=tvm-1392786932_3-20170712t132611z start_time=2017-07-12 14:04:43.585806+00:00 end_time=2017-07-12 14:16:18.403235+00:00 duration=0:11:34.817429 exit_code=0]
2017-07-12 14:17:58,285 INFO - job_id=cntk-parametricsweep-job task_id=dockertask-00001 [state=TaskState.completed max_retries=0 retention_time=10675199 days, 2:48:05.477581 pool_id=gpupool node_id=tvm-1392786932_2-20170712t132611z start_time=2017-07-12 14:04:43.643825+00:00 end_time=2017-07-12 14:12:44.257550+00:00 duration=0:08:00.613725 exit_code=0]
2017-07-12 14:17:58,285 INFO - job_id=cntk-parametricsweep-job task_id=dockertask-00002 [state=TaskState.completed max_retries=0 retention_time=10675199 days, 2:48:05.477581 pool_id=gpupool node_id=tvm-1392786932_1-20170712t132611z start_time=2017-07-12 14:04:43.751081+00:00 end_time=2017-

<a id='section4'></a>

# Check results
The results of our parameter search should now be saved to our output container.

**Note:** You will encounter errors if you did not wait for all tasks to complete with an exit code in the previous cell.

First let's alias `blobxfer` to aid in downloading our blobs. We will aggregate our results in the `MODELS_DIR`.

In [18]:
%alias blobxfer python -m blobxfer

MODELS_DIR = 'psmodels'

In [19]:
blobxfer $storage_account_name output $MODELS_DIR --remoteresource . --download --include "*_$JOB_ID/model_results.json" --storageaccountkey $storage_account_key

 azure blobxfer parameters [v0.12.1]
             platform: Linux-4.4.0-81-generic-x86_64-with-debian-stretch-sid
   python interpreter: CPython 2.7.11
     package versions: az.common=1.1.5 az.sml=0.20.5 az.stor=0.34.2 crypt=1.9 req=2.18.1
      subscription id: None
      management cert: None
   transfer direction: Azure->local
       local resource: psmodels
      include pattern: *_cntk-parametricsweep-job/model_results.json
      remote resource: .
   max num of workers: 24
              timeout: None
      storage account: batch0e43a94est
              use SAS: False
  upload as page blob: False
  auto vhd->page blob: False
 upload to file share: False
 container/share name: output
  container/share URI: https://batch0e43a94est.blob.core.windows.net/output
    compute block MD5: False
     compute file MD5: True
    skip on MD5 match: True
   chunk size (bytes): 4194304
     create container: True
  keep mismatched MD5: False
     recursive if dir: True
component strip on up: 1


Now we will combine all of the `model_results.json` files into one dictionary for analysis.

In [20]:
def scandir(basedir):
    for root, dirs, files in os.walk(basedir):
        for f in files:
            yield os.path.join(root, f) 

results_dict = {}
for model in scandir(MODELS_DIR):
    if not model.endswith('.json'):
        continue
    key = model.split(os.sep)[1]
    results_dict[key] = read_json(model)
    
print(json.dumps(results_dict, indent=4, sort_keys=True))

{
    "333db632_dockertask-00002_cntk-parametricsweep-job": {
        "parameters": {
            "max_epochs": 30, 
            "minibatch_size": 64, 
            "num_convolution_layers": 3
        }, 
        "test_metric": 0.2295
    }, 
    "59254dc2_dockertask-00000_cntk-parametricsweep-job": {
        "parameters": {
            "max_epochs": 30, 
            "minibatch_size": 32, 
            "num_convolution_layers": 3
        }, 
        "test_metric": 0.2163
    }, 
    "ce3e2a24_dockertask-00001_cntk-parametricsweep-job": {
        "parameters": {
            "max_epochs": 30, 
            "minibatch_size": 64, 
            "num_convolution_layers": 2
        }, 
        "test_metric": 0.2403
    }
}


From the aggregated results dictionary, we select the one with the smallest error:

In [21]:
tuple_min_error = min(results_dict.iteritems(), key=lambda x: x[1]['test_metric'])
configuration_with_min_error = tuple_min_error[0]
print('task with smallest error: {} ({})'.format(configuration_with_min_error, tuple_min_error[1]['test_metric']))

task with smallest error: 59254dc2_dockertask-00000_cntk-parametricsweep-job (0.2163)


<a id='section5'></a>

## Download best model
Now we'll download the corresponding best performing model.

In [22]:
MODEL_NAME = 'ConvNet_CIFAR10_model.dnn'
BEST_MODEL_BLOB_NAME = '{}/{}'.format(configuration_with_min_error, MODEL_NAME)
print(BEST_MODEL_BLOB_NAME)

59254dc2_dockertask-00000_cntk-parametricsweep-job/ConvNet_CIFAR10_model.dnn


In [23]:
blobxfer $storage_account_name output $MODELS_DIR --remoteresource $BEST_MODEL_BLOB_NAME --download --storageaccountkey $storage_account_key

 azure blobxfer parameters [v0.12.1]
             platform: Linux-4.4.0-81-generic-x86_64-with-debian-stretch-sid
   python interpreter: CPython 2.7.11
     package versions: az.common=1.1.5 az.sml=0.20.5 az.stor=0.34.2 crypt=1.9 req=2.18.1
      subscription id: None
      management cert: None
   transfer direction: Azure->local
       local resource: psmodels
      include pattern: None
      remote resource: 59254dc2_dockertask-00000_cntk-parametricsweep-job/ConvNet_CIFAR10_model.dnn
   max num of workers: 24
              timeout: None
      storage account: batch0e43a94est
              use SAS: False
  upload as page blob: False
  auto vhd->page blob: False
 upload to file share: False
 container/share name: output
  container/share URI: https://batch0e43a94est.blob.core.windows.net/output
    compute block MD5: False
     compute file MD5: True
    skip on MD5 match: True
   chunk size (bytes): 4194304
     create container: True
  keep mismatched MD5: False
     recursive if d

In [24]:
!mv $MODELS_DIR/$configuration_with_min_error/$MODEL_NAME $MODELS_DIR
!rm -rf $MODELS_DIR/*_$JOB_ID  # optionally remove all of the temporary result json directories/files
!ls -alF $MODELS_DIR

total 1920
drwxr-xr-x  2 nbuser nbuser    4096 Jul 12 14:18 ./
drwx------ 15 nbuser nbuser    4096 Jul 12 14:11 ../
-rw-r--r--  1 nbuser nbuser 1957354 Jul 12 14:18 ConvNet_CIFAR10_model.dnn


The best model file (`ConvNet_CIFAR10_model.dnn`) is now ready for use.

**Note:** We could have created a Batch task that did the model selection for us using task dependencies. The model selection task would be dependent upon all of the parametric sweep training tasks and would only run after those tasks complete successfully. The Batch task could then proceed with the logic above.

Please see the advanced notebook that shows how this is accomplished: [Automatic Model Selection from Parametric Sweep with Task Dependencies](05_Advanced_Auto_Model_Selection.ipynb)

<a id='section6'></a>

## Delete job

To delete the job use the command below. Just be aware that this will get rid of all the files created by the job and tasks.

In [25]:
shipyard jobs del -y --termtasks --wait

2017-07-12 14:18:23,129 INFO - Deleting job: cntk-parametricsweep-job
2017-07-12 14:18:23,129 DEBUG - disabling job cntk-parametricsweep-job first due to task termination
2017-07-12 14:18:23,974 DEBUG - Skipping termination of completed task dockertask-00000 on job cntk-parametricsweep-job
2017-07-12 14:18:24,182 DEBUG - Skipping termination of completed task dockertask-00001 on job cntk-parametricsweep-job
2017-07-12 14:18:24,378 DEBUG - Skipping termination of completed task dockertask-00002 on job cntk-parametricsweep-job
2017-07-12 14:18:24,737 DEBUG - waiting for job cntk-parametricsweep-job to delete
2017-07-12 14:18:56,482 INFO - job cntk-parametricsweep-job does not exist
